<a href="https://colab.research.google.com/github/rohankj01/My-work-Deep-into-CNNs/blob/main/W2-3/Competitions/Kaggle_Hackathon1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

{}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets list

ref                                                           title                                              size  lastUpdated          downloadCount  
------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                   Reddit Vaccine Myths                              229KB  2021-06-19 20:05:51           7886  
crowww/a-large-scale-fish-dataset                             A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           4790  
imsparsh/musicnet-dataset                                     MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1617  
dhruvildave/wikibooks-dataset                                 Wikibooks Dataset                                   1GB  2021-06-09 15:58:24           2261  
mathurinache/twitter-edge-nodes                               Tw

In [6]:
!kaggle competitions download -c tabular-playground-series-jun-2021

test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
! mkdir train
! unzip train.csv.zip -d train
! mkdir test
! unzip test.csv.zip -d test

mkdir: cannot create directory ‘train’: File exists
Archive:  train.csv.zip
replace train/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train/train.csv         
mkdir: cannot create directory ‘test’: File exists
Archive:  test.csv.zip
replace test/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test/test.csv           


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [9]:
train = pd.read_csv('train/train.csv')
test = pd.read_csv('test/test.csv')

In [10]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S


In [11]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,100000,3,"Holliday, Daniel",male,19.0,0,0,24745,63.01,NaN,S
1,100001,3,"Nguyen, Lorraine",female,53.0,0,0,13264,5.81,NaN,S
2,100002,1,"Harris, Heather",female,19.0,0,0,25990,38.91,B15315,C
3,100003,2,"Larsen, Eric",male,25.0,0,0,314011,12.93,NaN,S
4,100004,1,"Cleary, Sarah",female,17.0,0,2,26203,26.89,B22515,C


In [13]:
train = train.drop(['Name','Ticket','PassengerId'], axis=1)

In [14]:
one_hot_train1 = pd.concat([train, pd.get_dummies(train[['Embarked','Sex']])], axis=1)
one_hot_train2 = pd.concat([one_hot_train1, pd.get_dummies(train['Pclass'])], axis=1)
one_hot_train2['FamNo'] = one_hot_train2['SibSp'] + one_hot_train2['Parch']
one_hot_train2['Age'] = one_hot_train2['Age'].replace(np.nan, 39.00)
one_hot_train2['Age'] = one_hot_train2['Age']/87.00

one_hot_train2['Fare'] = one_hot_train2['Fare']/(one_hot_train2['Fare'].max())
one_hot_train2['FamNo'] = one_hot_train2['FamNo']/(one_hot_train2['FamNo'].max())
processed_data = one_hot_train2.drop(['Sex','Pclass','Embarked','Embarked_C','Sex_female','SibSp','Cabin','Parch',1], axis=1)

In [ ]:
processed_data.head()

In [17]:
test = test.drop(['Name','Ticket','PassengerId'], axis=1)

In [18]:
one_hot_test1 = pd.concat([test, pd.get_dummies(test[['Embarked','Sex']])], axis=1)
one_hot_test2 = pd.concat([one_hot_test1, pd.get_dummies(test['Pclass'])], axis=1)
one_hot_test2['FamNo'] = one_hot_test2['SibSp'] + one_hot_test2['Parch']
one_hot_test2['Age'] = one_hot_test2['Age'].replace(np.nan, 39.00)
one_hot_test2['Age'] = one_hot_test2['Age']/87.00

one_hot_test2['Fare'] = one_hot_test2['Fare']/(one_hot_test2['Fare'].max())
one_hot_test2['FamNo'] = one_hot_test2['FamNo']/(one_hot_test2['FamNo'].max())
processed_data_test = one_hot_test2.drop(['Sex','Pclass','Embarked','Embarked_C','Sex_female','SibSp','Cabin','Parch',1], axis=1)

In [ ]:
processed_data_test

In [20]:
features, targets = processed_data.drop(['Survived'], axis=1), processed_data['Survived']

In [22]:
# Activation (sigmoid) function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_prime(x):
    return sigmoid(x) * (1-sigmoid(x))
def error_formula(y, output):
    return - y*np.log(output) - (1 - y) * np.log(1-output)

In [23]:
def error_term_formula(x, y, output):
    error_term = (y-output)*sigmoid_prime(x)*x
    return error_term

In [ ]:
# Neural Network hyperparameters
epochs = 100
learnrate = 0.001

# Training function
def train_nn(features, targets, epochs, learnrate):
    
    # Use to same seed to make debugging easier
    np.random.seed(42)

    n_records, n_features = features.shape
    last_loss = None

    # Initialize weights
    weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

    for e in range(epochs):
        del_w = np.zeros(weights.shape)
        for x, y in zip(features.values, targets):
            # Loop through all records, x is the input, y is the target

            # Activation of the output unit
            #   Notice we multiply the inputs and the weights here 
            #   rather than storing h as a separate variable 
            output = sigmoid(np.dot(x, weights))

            # The error term
            error_term = error_term_formula(x, y, output)

            # The gradient descent step, the error times the gradient times the inputs
            del_w += error_term

        # Update the weights here. The learning rate times the 
        # change in weights
        # don't have to divide by n_records since it is compensated by the learning rate
        weights += learnrate * del_w #/ n_records  

        # Printing out the mean square error on the training set
        if e % (epochs / 10) == 0:
            out = sigmoid(np.dot(features, weights))
            loss = np.mean(error_formula(targets, out))
            print("Epoch:", e)
            if last_loss and last_loss < loss:
                print("Train loss: ", loss, "  WARNING - Loss Increasing")
            else:
                print("Train loss: ", loss)
            last_loss = loss
            print("=========")
    print("Finished training!")
    return weights
    
weights = train_nn(features, targets, epochs, learnrate)

In [ ]:
# #Loss Function
# def loss_function(y_pred,target):
#   loss = np.mean(-(target*np.log(y_pred) + (1-target)*np.log(1-y_pred)))
#   # print(loss)
#   return loss